# Н. Н. Голованов "Геометрическое моделирование"


In [128]:
import numpy as np
import matplotlib.pyplot as plt
import math
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
from sklearn.preprocessing import normalize
import plotly.graph_objects as go
import time

np.set_printoptions(precision=3)

E = np.identity(3)
LIM = 100
START = np.array([[0, 0, 0]]).T
axis = [
    [np.array([[-LIM, 0, 0]]).T, np.array([[LIM, 0, 0]]).T],
    [np.array([[0, -LIM, 0]]).T, np.array([[0, LIM, 0]]).T],
    [np.array([[0, 0, -LIM]]).T, np.array([[0, 0, LIM]]).T]
]

def frame_args(duration, from_current = False):
    return {
            "frame": {"duration": duration, "redraw": True},
            "mode": "immediate",
            "fromcurrent": from_current,
            "transition": {"duration": duration, "easing": "linear"},
        }
def empty():
    return go.Scatter3d(x=[2*LIM], y=[2*LIM], z=[2*LIM])
def scatter(p, color = "blue"):
    if isinstance(p, list):
        p = np.array(p)
    return go.Scatter3d(x=p[0], y=p[1], z=p[2], marker={"color": color})
def scatters(ps, mode = "lines+markers", color = "black"):
    if isinstance(ps, list):
        ps = np.array(ps)
    return go.Scatter3d(x=ps[:,0][:,0], y=ps[:,1][:,0], z=ps[:,2][:,0], marker={"color": color}, mode=mode)
def surface(n, q):
    X = np.linspace(-LIM, LIM, 100)
    Y = np.linspace(-LIM, LIM, 100)
    Z = [[(n[0, 0]*q[0, 0] + n[1, 0]*q[1, 0] + n[2, 0]*q[2, 0] - n[0, 0]*x - n[1, 0]*y)/n[2, 0] 
         for x in X] for y in Y]
    return go.Surface(x=X, y=Y, z=Z, opacity=0.5, showscale=False)
def plot_data(points = [], trace = True, mode = "markers", start=START):
    res = []
    for i in range(0, len(points)):
        if trace:
            res.append(go.Frame(data=[scatters([start, points[i]], color = "blue"), 
                                  scatters(points[:i], mode = mode, color = "blue")]))
        else:
            res.append(go.Frame(data=[scatters([start, points[i]], mode=mode, color = "blue")]))
    return res

\textbf{Поворот точки вокруг оси в пространстве}

Дано: $q$ - радиус-вектор оси вращения, $v$ - орт, $r_0$ - начальная точка на окружности, $\alpha$ - угол поворота

\begin{equation*}
r = q + A(r_0 - q)
\end{equation*}

\begin{equation*}
A = v v^T(1 - \cos \alpha) + E \cos \alpha + V_x \sin \alpha
\end{equation*}

\begin{equation*}
v v^T = \begin{bmatrix} 
v_1v_1 & v_1v_2 & v_1v_3 \\ 
v_2v_1 & v_2v_2 & v_2v_3 \\ 
v_3v_1 & v_3v_2 & v_3v_3 \\ 
\end{bmatrix}
\end{equation*}

\begin{equation*}
E = \begin{bmatrix} 
1 & 0 & 0 \\ 
0 & 1 & 0 \\ 
0 & 0 & 1 \\ 
\end{bmatrix}
\end{equation*}

\begin{equation*}
V_x = \begin{bmatrix} 
0 & -v_3 & v_2 \\ 
v_2 & 0 & -v_1 \\ 
-v_2 & v_1 & 0 \\ 
\end{bmatrix}
\end{equation*}

При $v = \begin{bmatrix} 1 \\ 0 \\ 0 \end{bmatrix}$, $v = \begin{bmatrix} 0 \\ 1 \\ 0 \end{bmatrix}$, $v = \begin{bmatrix} 0 \\ 0 \\ 1 \end{bmatrix}$ имеем матрицы поворота вокруг осей $x, y, z$:
\begin{equation*}
A = \begin{bmatrix} 
1 & 0 & 0 \\ 
0 & \cos \alpha & -\sin \alpha \\ 
0 & \sin \alpha & \cos \alpha \\ 
\end{bmatrix}
\end{equation*}

\begin{equation*}
A = \begin{bmatrix} 
\cos \alpha & 0 & \sin \alpha \\ 
0 & 1 & 0 \\ 
-\sin \alpha & 0 & \cos \alpha \\ 
\end{bmatrix}
\end{equation*}

\begin{equation*}
A = \begin{bmatrix} 
\cos \alpha & -\sin \alpha & 0 \\ 
\sin \alpha & \cos \alpha & 0 \\ 
0 & 0 & 1 \\ 
\end{bmatrix}
\end{equation*}

In [129]:
q = np.array([[30, 30, 5]]).T
v = normalize(np.array([[-1, 5, 5]])).T
r0 = np.array([[50, -50, 10]]).T

Vx = np.array([
    [0, -v[2,0], v[1,0]],
    [v[2,0], 0, -v[0,0]],
    [-v[1,0], v[0,0], 0]
])

points = []

for alpha in np.arange(0, 2*math.pi, 0.1):
    A = v * v.T * (1 - math.cos(alpha)) + E * math.cos(alpha) + Vx * math.sin(alpha)
    r = q + A.dot(r0 - q)
    points.append(r)

points = np.array(points)

fig = go.Figure(
    data=[scatters([START, points[0]], color = "blue"), scatter(START), 
          scatters(axis[0], "lines"), scatters(axis[1], "lines"), scatters(axis[2], "lines")])
fig.update_layout(
        scene = dict(
            xaxis=dict(range=[-LIM, LIM], autorange=False),
            yaxis=dict(range=[-LIM, LIM], autorange=False),
            zaxis=dict(range=[-LIM, LIM], autorange=False),
            aspectratio=dict(x=1, y=1, z=1),
        ),
        showlegend=False,
        width=800,
        height=800,
        updatemenus=[dict(
            type="buttons",
            buttons=[
                    {
                        "args": [None, frame_args(5, True)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },]
        )])
fig.frames = plot_data(points)

fig.show()

\textbf{Симметрия точки относительно плоскости}

Дано: $q$ - радиус-вектор точки на плоскости, $v$, $u$ - орты плоскости, $r_0$ - начальная точка

\begin{equation*}
r = q + A(r_0 - q)
\end{equation*}

\begin{equation*}
A = 2uu^T + 2vv^T - E
\end{equation*}

\begin{equation*}
u u^T = \begin{bmatrix} 
u_1u_1 & u_1u_2 & u_1u_3 \\ 
u_2u_1 & u_2u_2 & u_2u_3 \\ 
u_3u_1 & u_3u_2 & u_3u_3 \\ 
\end{bmatrix}
\end{equation*}

\begin{equation*}
v v^T = \begin{bmatrix} 
v_1v_1 & v_1v_2 & v_1v_3 \\ 
v_2v_1 & v_2v_2 & v_2v_3 \\ 
v_3v_1 & v_3v_2 & v_3v_3 \\ 
\end{bmatrix}
\end{equation*}

In [130]:
q = np.array([[3, 9, -5]]).T
v = normalize(np.array([[3, -9, 5]])).T
u = normalize(np.array([[-1, -7, 5]])).T
r0 = np.array([[35, -5, 40]]).T

v = normalize(np.cross(np.cross(u.T, v.T), u.T)).T

points = []
A = 2*(u*u.T + v*v.T) - E
r = q + A.dot(r0 - q)
points.append(r0)
points.append(r)

points = np.array(points)

fig = go.Figure(
    data=[empty(), empty(), scatters([q, r0], mode="markers", color = "blue"), 
          scatters([q - LIM*u, q + LIM*u], color="gray", mode="lines"),
          scatters([q - LIM*v, q + LIM*v], color="gray", mode="lines"),
          surface(np.cross(u.T, v.T).T, q),
          scatters(axis[0], "lines"), scatters(axis[1], "lines"), scatters(axis[2], "lines")])
fig.update_layout(
        scene = dict(
            xaxis=dict(range=[-LIM, LIM], autorange=False),
            yaxis=dict(range=[-LIM, LIM], autorange=False),
            zaxis=dict(range=[-LIM, LIM], autorange=False),
            aspectratio=dict(x=1, y=1, z=1),
        ),
        showlegend=False,
        width=800,
        height=800,
        updatemenus=[dict(
            type="buttons",
            buttons=[
                    {
                        "args": [None, frame_args(5, True)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },]
        )])
fig.frames = plot_data(points, mode = "markers+lines", start=r0)

fig.show()

\textbf{Масштабирование в пространстве}

Дано: $q$ - радиус-вектор, $r_0$ - начальная точка, $m$ - вектор масштабирования

\begin{equation*}
r = q + A(r_0 - q)
\end{equation*}

\begin{equation*}
A = \begin{bmatrix} 
m_1 & 0 & 0 \\ 
0 & m_2 & 0 \\ 
0 & 0 & m_3 \\ 
\end{bmatrix}
\end{equation*}

In [131]:
q = np.array([[-5, 30, 5]]).T
r0 = np.array([[10, 20, 10]]).T
m = np.array([[5, 5, 5]]).T

points = []

A = np.array([[m[0, 0], 0, 0], [0, m[1, 0], 0], [0, 0, m[2, 0]]])
r = q + A.dot(r0 - q)
points.append(r0)
points.append(r)

points = np.array(points)

fig = go.Figure(
    data=[scatters([q, r0], color = "blue"), empty(), empty(),
          scatters(axis[0], "lines"), scatters(axis[1], "lines"), scatters(axis[2], "lines")])
fig.update_layout(
        scene = dict(
            xaxis=dict(range=[-LIM, LIM], autorange=False),
            yaxis=dict(range=[-LIM, LIM], autorange=False),
            zaxis=dict(range=[-LIM, LIM], autorange=False),
            aspectratio=dict(x=1, y=1, z=1),
        ),
        showlegend=False,
        width=800,
        height=800,
        updatemenus=[dict(
            type="buttons",
            buttons=[
                    {
                        "args": [None, frame_args(5, True)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },]
        )])
fig.frames = plot_data(points, start=q)

fig.show()

\textbf{Симметрия точки относительно линии}

Дано: $q$ - радиус-вектор точки на линии, $v$ - орт линии, $r_0$ - начальная точка

\begin{equation*}
r = q + A(r_0 - q)
\end{equation*}

\begin{equation*}
A = 2uu^T - E
\end{equation*}

\begin{equation*}
u u^T = \begin{bmatrix} 
u_1u_1 & u_1u_2 & u_1u_3 \\ 
u_2u_1 & u_2u_2 & u_2u_3 \\ 
u_3u_1 & u_3u_2 & u_3u_3 \\ 
\end{bmatrix}
\end{equation*}

In [132]:
q = np.array([[3, 0, 5]]).T
u = normalize(np.array([[2, -5, 4]])).T
r0 = np.array([[-50, -5, 10]]).T

points = []

A = 2*(u*u.T) - E
r = q + A.dot(r0 - q)
points.append(r0)
points.append(r)

points = np.array(points)

fig = go.Figure(
    data=[empty(), empty(), scatters([q, r0], color = "blue", mode="markers"), 
          scatters([q - LIM*u, q + LIM*u], color="gray", mode="lines"),
          scatters(axis[0], "lines"), scatters(axis[1], "lines"), scatters(axis[2], "lines")])
fig.update_layout(
        scene = dict(
            xaxis=dict(range=[-LIM, LIM], autorange=False),
            yaxis=dict(range=[-LIM, LIM], autorange=False),
            zaxis=dict(range=[-LIM, LIM], autorange=False),
            aspectratio=dict(x=1, y=1, z=1),
        ),
        showlegend=False,
        width=800,
        height=800,
        updatemenus=[dict(
            type="buttons",
            buttons=[
                    {
                        "args": [None, frame_args(5, True)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },]
        )])
fig.frames = plot_data(points, mode = "markers+lines", start=r0)

fig.show()